In [25]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Adam
import torchvision.models as models
from sklearn.metrics import confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt

In [26]:
torch.manual_seed(42)
np.random.seed(42)

In [27]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("The device is", device)

The device is cpu


# Task 1

### Preparing the Data

In [28]:
# Define transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

In [10]:
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [29]:
print(f'Full training set size: {len(full_trainset)}')
print(f'Test set size: {len(testset)}')
print(f'Classes: {full_trainset.classes}')

Full training set size: 50000
Test set size: 10000
Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [30]:
## Create Balanced Subset (1000 images per class)

def create_balanced_subset(dataset, samples_per_class=1000, random_seed=42):
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    
    # Get all data and targets
    data = []
    targets = []
    
    for i in range(len(dataset)):
        img, label = dataset[i]
        data.append(img)
        targets.append(label)
    
    data = torch.stack(data)
    targets = torch.tensor(targets)
    
    # Create balanced subset
    subset_data = []
    subset_targets = []
    
    num_classes = len(dataset.classes)
    
    for class_idx in range(num_classes):
        # Find all indices for this class
        class_indices = torch.where(targets == class_idx)[0]
        
        # Randomly select samples_per_class indices
        selected_indices = torch.randperm(len(class_indices))[:samples_per_class]
        selected_indices = class_indices[selected_indices]
        
        # Add selected samples to subset
        subset_data.append(data[selected_indices])
        subset_targets.extend([class_idx] * samples_per_class)
    
    # Combine all classes
    subset_data = torch.cat(subset_data, dim=0)
    subset_targets = torch.tensor(subset_targets)
    
    return subset_data, subset_targets

In [31]:
train_images, train_targets = create_balanced_subset(full_trainset, samples_per_class=1000)

In [32]:
# Extract test data
test_data = []
test_labels = []
for i in range(len(testset)):
    img, label = testset[i]
    test_data.append(img)
    test_labels.append(label)

test_images = torch.stack(test_data)
test_targets = torch.tensor(test_labels)

print(f'Training subset size: {train_images.shape}')
print(f'Test set size: {test_images.shape}')

Training subset size: torch.Size([10000, 3, 32, 32])
Test set size: torch.Size([10000, 3, 32, 32])


In [33]:
# Verify class distribution
unique_classes, counts = torch.unique(train_targets, return_counts=True)
print(f'Class distribution in training subset:')
for class_idx, count in zip(unique_classes, counts):
    print(f'  Class {class_idx} ({full_trainset.classes[class_idx]}): {count} samples')

Class distribution in training subset:
  Class 0 (airplane): 1000 samples
  Class 1 (automobile): 1000 samples
  Class 2 (bird): 1000 samples
  Class 3 (cat): 1000 samples
  Class 4 (deer): 1000 samples
  Class 5 (dog): 1000 samples
  Class 6 (frog): 1000 samples
  Class 7 (horse): 1000 samples
  Class 8 (ship): 1000 samples
  Class 9 (truck): 1000 samples


## Defining the Dataset 

In [34]:
class CIFAR10Dataset(Dataset):
    def __init__(self, x, y):
        # Data is already normalized and in tensor format
        self.x, self.y = x, y
    
    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]
        return x.to(device), y.to(device)
    
    def __len__(self):
        return len(self.x)

In [35]:
def get_training_data():
    train = CIFAR10Dataset(train_images, train_targets)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True)
    return trn_dl

def get_test_data():
    test = CIFAR10Dataset(test_images, test_targets)
    test_dl = DataLoader(test, batch_size=32, shuffle=False)
    return test_dl

# Task 2

#### Define Custom Neural Network, Loss function and Optimizer

In [41]:
def get_cnn():
    model = nn.Sequential(
        # First convolutional block
        nn.Conv2d(3, 32, kernel_size=3, padding=1),  # 32x32x32
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 32, kernel_size=3, padding=1), # 32x32x32
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),       # 16x16x32
        nn.Dropout2d(0.25),
        
        # Second convolutional block
        nn.Conv2d(32, 64, kernel_size=3, padding=1), # 16x16x64
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, padding=1), # 16x16x64
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),       # 8x8x64
        nn.Dropout2d(0.25),
        
        # Third convolutional block
        nn.Conv2d(64, 128, kernel_size=3, padding=1), # 8x8x128
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, padding=1), # 8x8x128
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),        # 4x4x128
        nn.Dropout2d(0.25),
        
        # Fully connected layers
        nn.Flatten(),
        nn.Linear(128 * 4 * 4, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 10)
    ).to(device)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

# Task 3

#### Load and Adapt ModelNetV2

In [37]:
def get_mobilenetv2():
    # Load pretrained MobileNetV2
    model = models.mobilenet_v2(pretrained=True)
    
    # Modify the classifier for CIFAR-10 (10 classes)
    # MobileNetV2's classifier is a single linear layer
    num_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(num_features, 10)
    )
    
    # Move model to device
    model = model.to(device)
    
    # Properly initialize the new classifier layer
    nn.init.xavier_uniform_(model.classifier[1].weight)
    nn.init.zeros_(model.classifier[1].bias)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

# Task 4

In [38]:
## Training Function
def train_batch(x, y, model, optimizer, loss_fn):
    model.train()  # Set the model to the "train" mode
    
    # Apply the model to the inputs
    prediction = model(x)
    
    # Compute loss
    batch_loss = loss_fn(prediction, y)
    
    # Based on the forward pass in `model(x)`, compute all the
    # gradients of 'model.parameters()'
    batch_loss.backward()
    
    # Apply new-weights = f(old-weights, old-weight-gradients)
    # where "f" is the optimizer
    optimizer.step()
    
    # Flush gradients memory for next batch of calculations
    optimizer.zero_grad()
    
    return batch_loss.item()

In [39]:
def accuracy(x, y, model):
    model.eval()  # Set the model to the "evaluation" mode
    
    # Get the prediction matrix for a tensor of `x` images
    prediction = model(x)
    
    # Compute if the location of maximum in each row
    # coincides with ground truth
    max_values, argmaxes = prediction.max(-1)
    
    # Compute the accuracy by checking if the locations of maximum values
    # coincide with the ground truth values `y`
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [40]:
def train_model(model, optimizer, loss_fn, train_dl, num_epochs=10, model_name="Model"):
    """
    Modular training function that works for either model
    Uses the same hyperparameters for both models
    """
    print(f"\nTraining {model_name}...")
    
    # Initialize the lists that will contain the accuracy and loss values
    accuracies = []
    losses = []
    
    for epoch in range(num_epochs):
        print(f"Epoch: {epoch}")
        
        # Initialize the lists for each batch
        epoch_accuracies = []
        epoch_losses = []
        
        # Iterate for each batch
        for batch in train_dl:
            x, y = batch
            # Compute the loss and accuracy
            batch_loss = train_batch(x, y, model, optimizer, loss_fn)
            batch_accuracy = accuracy(x, y, model)
            epoch_losses.append(batch_loss)
            epoch_accuracies.extend(batch_accuracy)
        
        # Store the mean loss and accuracy within an epoch
        epoch_loss = np.mean(epoch_losses)
        epoch_accuracy = np.mean(epoch_accuracies)
        losses.append(epoch_loss)
        accuracies.append(epoch_accuracy)
        
        print(f"  Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")
    
    return model, accuracies, losses

In [ ]:
# Get data loaders
train_dl = get_training_data()
test_dl = get_test_data()

# Training parameters (same hyperparameters for both models)
num_epochs = 10

# Train both models on the same training subset
print("="*50)
print("TRAINING CUSTOM CNN")
print("="*50)
custom_model, custom_loss_fn, custom_optimizer = get_cnn()
custom_model, custom_accuracies, custom_losses = train_model(
    custom_model, custom_optimizer, custom_loss_fn, train_dl, num_epochs, "Custom CNN"
)


TRAINING CUSTOM CNN

Training Custom CNN...
Epoch: 0
  Train Loss: 1.8174, Train Accuracy: 0.3818
Epoch: 1
  Train Loss: 1.5345, Train Accuracy: 0.5162
Epoch: 2
  Train Loss: 1.3877, Train Accuracy: 0.5761
Epoch: 3
  Train Loss: 1.2546, Train Accuracy: 0.6375
Epoch: 4
  Train Loss: 1.1472, Train Accuracy: 0.6810
Epoch: 5
  Train Loss: 1.0614, Train Accuracy: 0.7170
Epoch: 6
  Train Loss: 0.9934, Train Accuracy: 0.7486
Epoch: 7
  Train Loss: 0.9263, Train Accuracy: 0.7643
Epoch: 8
  Train Loss: 0.8671, Train Accuracy: 0.7949
Epoch: 9
  Train Loss: 0.8253, Train Accuracy: 0.8195

TRAINING MOBILENETV2


D:\anaconda\envs\my_pytorch_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\anaconda\envs\my_pytorch_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\Admin/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100.0%



Training MobileNetV2...
Epoch: 0
  Train Loss: 1.6348, Train Accuracy: 0.5463
Epoch: 1
  Train Loss: 1.1827, Train Accuracy: 0.6850
Epoch: 2


In [ ]:
print("\n" + "="*50)
print("TRAINING MOBILENETV2")
print("="*50)
mobilenet_model, mobilenet_loss_fn, mobilenet_optimizer = get_mobilenetv2()
mobilenet_model, mobilenet_accuracies, mobilenet_losses = train_model(
    mobilenet_model, mobilenet_optimizer, mobilenet_loss_fn, train_dl, num_epochs, "MobileNetV2"
)

In [ ]:
## Test the Model

test_dl = get_test_data()
test_accuracies = []
test_losses = []

for batch in test_dl:
    x, y = batch
    batch_accuracy = accuracy(x, y, model)
    test_accuracies.extend(batch_accuracy)

test_accuracy = np.mean(test_accuracies)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Final Train Accuracy: {accuracies[-1]:.4f}')


In [ ]:
# Plot training progress comparison
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(range(num_epochs), custom_losses, label='Custom CNN', marker='o')
plt.plot(range(num_epochs), mobilenet_losses, label='MobileNetV2', marker='s')
plt.title('Training Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(num_epochs), custom_accuracies, label='Custom CNN', marker='o')
plt.plot(range(num_epochs), mobilenet_accuracies, label='MobileNetV2', marker='s')
plt.title('Training Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
#Task 5

In [ ]:
def evaluate_model(model, test_dl, model_name):
    """
    Evaluate a model on the full CIFAR-10 test set
    Report overall test accuracy clearly
    """
    test_accuracies = []
    all_predictions = []
    all_targets = []
    
    for batch in test_dl:
        x, y = batch
        batch_accuracy = accuracy(x, y, model)
        test_accuracies.extend(batch_accuracy)
        
        # Get predictions for confusion matrix
        model.eval()
        with torch.no_grad():
            outputs = model(x)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(y.cpu().numpy())
    
    test_accuracy = np.mean(test_accuracies)
    print(f'{model_name} Test Accuracy: {test_accuracy:.4f}')
    
    return test_accuracy, all_predictions, all_targets

In [ ]:
# Evaluate both models on the full CIFAR-10 test set
print("\n" + "="*50)
print("MODEL EVALUATION RESULTS")
print("="*50)

custom_test_acc, custom_preds, custom_targets = evaluate_model(
    custom_model, test_dl, "Custom CNN"
)

mobilenet_test_acc, mobilenet_preds, mobilenet_targets = evaluate_model(
    mobilenet_model, test_dl, "MobileNetV2"
)

print(f"\nFinal Custom CNN Train Accuracy: {custom_accuracies[-1]:.4f}")
print(f"Final MobileNetV2 Train Accuracy: {mobilenet_accuracies[-1]:.4f}")


In [ ]:
# Task 6

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes, model_name):
    """
    Generate and display confusion matrices using sklearn
    Ensure axes are labeled and readable
    """
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Calculate per-class accuracy
    per_class_acc = cm.diagonal() / cm.sum(axis=1)
    print(f"\n{model_name} - Per-class accuracies:")
    for i, class_name in enumerate(classes):
        print(f"  {class_name}: {per_class_acc[i]:.3f}")

In [ ]:
# Generate and display confusion matrices for both models
print("\n" + "="*50)
print("CONFUSION MATRICES")
print("="*50)

plot_confusion_matrix(custom_targets, custom_preds, full_trainset.classes, "Custom CNN")
plot_confusion_matrix(mobilenet_targets, mobilenet_preds, full_trainset.classes, "MobileNetV2")

In [ ]:
## Final Results Summary

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)
print(f"Custom CNN:")
print(f"  - Parameters: {count_parameters(custom_model):,}")
print(f"  - Final Train Accuracy: {custom_accuracies[-1]:.4f}")
print(f"  - Test Accuracy: {custom_test_acc:.4f}")

print(f"\nMobileNetV2:")
print(f"  - Parameters: {count_parameters(mobilenet_model):,}")
print(f"  - Final Train Accuracy: {mobilenet_accuracies[-1]:.4f}")
print(f"  - Test Accuracy: {mobilenet_test_acc:.4f}")

accuracy_diff = mobilenet_test_acc - custom_test_acc
print(f"\nMobileNetV2 advantage: {accuracy_diff:+.4f} ({accuracy_diff*100:+.2f}%)")


In [ ]:
# Task 8

In [ ]:
print("\n" + "="*60)
print("TASK 8: PERFORMANCE ANALYSIS")
print("="*60)

# Compare test accuracy
print("1. TEST ACCURACY COMPARISON:")
print(f"   Custom CNN Test Accuracy: {custom_test_acc:.4f} ({custom_test_acc*100:.2f}%)")
print(f"   MobileNetV2 Test Accuracy: {mobilenet_test_acc:.4f} ({mobilenet_test_acc*100:.2f}%)")
print(f"   Difference: {accuracy_diff:+.4f} ({accuracy_diff*100:+.2f}%)")

# Analyze training stability and convergence
print("\n2. TRAINING STABILITY AND CONVERGENCE:")
custom_loss_std = np.std(custom_losses)
mobilenet_loss_std = np.std(mobilenet_losses)
custom_acc_improvement = custom_accuracies[-1] - custom_accuracies[0]
mobilenet_acc_improvement = mobilenet_accuracies[-1] - mobilenet_accuracies[0]

print(f"   Custom CNN:")
print(f"     - Loss standard deviation: {custom_loss_std:.4f}")
print(f"     - Accuracy improvement: {custom_acc_improvement:.4f}")
print(f"     - Final loss: {custom_losses[-1]:.4f}")
print(f"   MobileNetV2:")
print(f"     - Loss standard deviation: {mobilenet_loss_std:.4f}")
print(f"     - Accuracy improvement: {mobilenet_acc_improvement:.4f}")
print(f"     - Final loss: {mobilenet_losses[-1]:.4f}")

# Analyze generalization
print("\n3. GENERALIZATION TO UNSEEN DATA:")
custom_overfitting = custom_accuracies[-1] - custom_test_acc
mobilenet_overfitting = mobilenet_accuracies[-1] - mobilenet_test_acc

print(f"   Custom CNN:")
print(f"     - Train-Test gap: {custom_overfitting:+.4f}")
print(f"     - Generalization: {'Good' if abs(custom_overfitting) < 0.05 else 'Moderate' if abs(custom_overfitting) < 0.1 else 'Poor'}")
print(f"   MobileNetV2:")
print(f"     - Train-Test gap: {mobilenet_overfitting:+.4f}")
print(f"     - Generalization: {'Good' if abs(mobilenet_overfitting) < 0.05 else 'Moderate' if abs(mobilenet_overfitting) < 0.1 else 'Poor'}")

# Discuss trade-offs
print("\n4. TRADE-OFFS (COMPLEXITY VS. PERFORMANCE):")
param_ratio = count_parameters(mobilenet_model) / count_parameters(custom_model)
print(f"   Parameter ratio (MobileNet/Custom): {param_ratio:.1f}x")
print(f"   Custom CNN: Simpler architecture, fewer parameters, faster training")
print(f"   MobileNetV2: Pretrained features, more complex, better performance")


In [ ]:
# Task 9

In [ ]:
print("\n" + "="*60)
print("TASK 9: MISCLASSIFIED CASE ANALYSIS")
print("="*60)

def analyze_misclassifications(y_true, y_pred, test_images, classes, model_name, num_samples=8):
    """
    Identify and visualize misclassified test samples
    Analyze why these images may have been incorrectly classified
    """
    # Find misclassified indices
    misclassified_indices = []
    for i, (true_label, pred_label) in enumerate(zip(y_true, y_pred)):
        if true_label != pred_label:
            misclassified_indices.append(i)
    
    print(f"\n{model_name} Misclassification Analysis:")
    print(f"Total misclassified: {len(misclassified_indices)} out of {len(y_true)}")
    print(f"Error rate: {len(misclassified_indices)/len(y_true)*100:.2f}%")
    
    # Show sample misclassifications
    plt.figure(figsize=(16, 8))
    sample_indices = np.random.choice(misclassified_indices, min(num_samples, len(misclassified_indices)), replace=False)
    
    for i, idx in enumerate(sample_indices):
        plt.subplot(2, 4, i + 1)
        # Convert from normalized tensor to displayable image
        img = test_images[idx].permute(1, 2, 0)  # Change from CHW to HWC
        img = img * 0.5 + 0.5  # Denormalize from [-1,1] to [0,1]
        img = torch.clamp(img, 0, 1)  # Ensure values are in [0,1]
        plt.imshow(img)
        
        true_label = classes[y_true[idx]]
        pred_label = classes[y_pred[idx]]
        plt.title(f'True: {true_label}\nPred: {pred_label}', color='red')
        plt.axis('off')
    
    plt.suptitle(f'{model_name} - Misclassified Samples', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Analyze confusion patterns
    cm = confusion_matrix(y_true, y_pred)
    print(f"\nMost confused class pairs for {model_name}:")
    
    # Find top confusion pairs (excluding diagonal)
    confusion_pairs = []
    for i in range(len(classes)):
        for j in range(len(classes)):
            if i != j and cm[i][j] > 0:
                confusion_pairs.append((i, j, cm[i][j]))
    
    # Sort by confusion count
    confusion_pairs.sort(key=lambda x: x[2], reverse=True)
    
    for i, (true_class, pred_class, count) in enumerate(confusion_pairs[:5]):
        print(f"  {i+1}. {classes[true_class]} → {classes[pred_class]}: {count} cases")

# Analyze misclassifications for both models
analyze_misclassifications(custom_targets, custom_preds, test_images, full_trainset.classes, "Custom CNN")
analyze_misclassifications(mobilenet_targets, mobilenet_preds, test_images, full_trainset.classes, "MobileNetV2")

print("\nVISUAL SIMILARITY ANALYSIS:")
print("Common confusion patterns often occur between visually similar classes:")
print("- Cat ↔ Dog: Similar fur textures and poses")
print("- Automobile ↔ Truck: Both are vehicles with similar shapes")
print("- Bird ↔ Airplane: Both can appear in sky with similar silhouettes")
print("- Deer ↔ Horse: Similar animal body structures")

In [ ]:
# Task 10

In [ ]:
print("\n" + "="*60)
print("TASK 10: EFFICIENCY COMMENTARY")
print("="*60)

# Model size comparison
custom_params = count_parameters(custom_model)
mobilenet_params = count_parameters(mobilenet_model)

print("1. MODEL SIZE (Number of Parameters):")
print(f"   Custom CNN: {custom_params:,} parameters")
print(f"   MobileNetV2: {mobilenet_params:,} parameters")
print(f"   Size ratio: {mobilenet_params/custom_params:.1f}x larger")

# Inference speed analysis
print("\n2. INFERENCE SPEED:")
print("   Custom CNN:")
print("     - Smaller model = faster inference")
print("     - Simple architecture = less computational overhead")
print("     - Suitable for real-time applications")
print("   MobileNetV2:")
print("     - Larger model = slower inference")
print("     - More complex operations (depthwise separable convolutions)")
print("     - Still relatively efficient compared to other pretrained models")

# Measure actual inference time
import time

def measure_inference_time(model, test_dl, num_batches=10):
    """Measure average inference time per batch"""
    model.eval()
    times = []
    
    with torch.no_grad():
        for i, batch in enumerate(test_dl):
            if i >= num_batches:
                break
            x, y = batch
            
            start_time = time.time()
            _ = model(x)
            end_time = time.time()
            
            times.append(end_time - start_time)
    
    return np.mean(times)

custom_inference_time = measure_inference_time(custom_model, test_dl)
mobilenet_inference_time = measure_inference_time(mobilenet_model, test_dl)

print(f"\n   Measured inference times (per batch of 32 images):")
print(f"   Custom CNN: {custom_inference_time*1000:.2f} ms")
print(f"   MobileNetV2: {mobilenet_inference_time*1000:.2f} ms")
print(f"   Speed ratio: {mobilenet_inference_time/custom_inference_time:.1f}x slower")

print("\n3. SUITABILITY FOR EDGE DEVICES AND REAL-TIME APPLICATIONS:")
print("   Custom CNN:")
print("     ✓ Lightweight and fast")
print("     ✓ Low memory footprint")
print("     ✓ Excellent for edge devices")
print("     ✓ Real-time capable")
print("     ✗ Lower accuracy")

print("   MobileNetV2:")
print("     ✓ Higher accuracy")
print("     ✓ Pretrained features")
print("     ✓ Still relatively efficient (designed for mobile)")
print("     ⚠ Larger model size")
print("     ⚠ Higher computational requirements")
print("     ⚠ May require optimization for real-time edge deployment")

print("\nRECOMMENDATIONS:")
print("- Use Custom CNN for: Resource-constrained environments, real-time applications")
print("- Use MobileNetV2 for: Applications where accuracy is critical, sufficient computational resources available")